In [1]:
import os
import tqdm
import pandas as pd
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.utils as utils
from torch.utils.data import Dataset, DataLoader

import Set
import Utility
import SQLSentence
import Model

%load_ext autoreload
%autoreload 1
%aimport Model
%aimport Set
%aimport Utility

In [2]:
# Build Environment
connect = Utility.connect_to_database()
main_data, list_stock_type, dict_code_name, dict_type_name = Utility.fetch_data_from_db()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Constants
PATH_CHECKPOINT = 'checkpoint'

if not os.path.exists(PATH_CHECKPOINT):
    os.makedirs(PATH_CHECKPOINT)

myseed = 42069  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [3]:
# Some Functions

# Dataset
class StockDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.shape = data.shape

    def __getitem__(self, index):
        data = torch.tensor(self.data[index], dtype=torch.float32)
        label = torch.tensor(self.label[index], dtype=torch.float32)

        return data, label
    
    def __len__(self):
        return len(self.data)
    
# Invoke Model
def invoke_model(mode, model, dataloader, criterion, device, optimizer=None, threshold=0.0):
    
    model.train() if mode == 'train' else model.eval()

    total_loss = 0
    correct, accuracy_count = 0, 0
    type1_error, type1_error_count = 0, 0

    for input, label in dataloader:
        input, label = input.to(device), label.to(device)

        if mode == 'train' and optimizer is not None:
            optimizer.zero_grad()
        
        outputs = model(input)
        loss = criterion(outputs.squeeze(1), label)
        
        total_loss+=loss.detach().cpu().item()
        
        predicted = (outputs.squeeze(1) > threshold).float()

        if mode == 'train' and optimizer is not None:
            loss.backward()
            optimizer.step()

        correct += (predicted == label).sum().item()
        accuracy_count += label.size(0)

        t1_error, t1_error_cnt = calculate_correct_count(predicted, label)


        type1_error += t1_error
        type1_error_count += t1_error_cnt

    avg_loss = round(total_loss/len(dataloader), 6)
    accuracy =  round((correct/accuracy_count)*100 , 2)

    if type1_error_count == 0:
        type1_error_count = 1

    type1_error_ratio = round((type1_error/type1_error_count)*100, 2)

    print(f'{mode} Loss: {avg_loss}, Accuracy: {accuracy}%, Type1 Error Ratio: {type1_error_ratio}%')

    return avg_loss, type1_error_ratio

def process_stock_data(stock_data, target, window_length, predict = False):
    stock_data = stock_data.drop(['code', 'date'], axis=1).reset_index(drop=True)
    stock_data = Set.Feature(stock_data)
    stock_data = stock_data.dropna()

    index_col_open = stock_data.columns.get_loc("open")
    index_col_high = stock_data.columns.get_loc("high")
    index_col_low = stock_data.columns.get_loc("low")
    index_col_volume = stock_data.columns.get_loc("volume")

    stock_data = stock_data.to_numpy()

    data_merged = []
    label_merged = []

    if predict:
        return stock_data[-window_length:, index_col_volume + 1:]

    for i in range(len(stock_data) - window_length):
        data = stock_data[i:i + window_length, index_col_volume + 1:]

        if target >= 0:
            target_value = (stock_data[i + window_length, index_col_high] - stock_data[i + window_length, index_col_open]) / \
                stock_data[i + window_length, index_col_open]
            label = int(target_value >= target)
        else:
            target_value = (stock_data[i + window_length, index_col_low] - stock_data[i + window_length, index_col_open]) / \
                stock_data[i + window_length, index_col_open]
            label = int(target_value <= target)

        data_merged.append(data)
        label_merged.append(label)

    return np.array(data_merged), np.array(label_merged)

def build_dataset(main_data, code_list, target, window_length):
    all_train_data = []
    all_test_data = []

    all_train_label = []
    all_test_label = []

    for code in code_list:
        stock_data = main_data[main_data['code'] == code].copy()

        data, label = process_stock_data(stock_data, target, window_length)
        train_size = int(0.9 * len(data))
               
        train_data = data[:train_size]
        test_data = data[train_size:]

        train_label = label[:train_size]
        test_label = label[train_size:]

        all_train_data.extend(train_data)
        all_test_data.extend(test_data)

        all_train_label.extend(train_label)
        all_test_label.extend(test_label)
    
    return np.array(all_train_data), np.array(all_test_data), np.array(all_train_label), np.array(all_test_label)

def build_dataloader(main_data, code_list, target, batch_size, window_length):
    train_data, test_data, train_label, test_label = build_dataset(main_data, code_list, target, window_length)
    dataset_train = StockDataset(train_data, train_label)
    dataset_test = StockDataset(test_data, test_label)
    datalaoder_train = DataLoader(dataset_train, batch_size, shuffle=True)
    datalaoder_test = DataLoader(dataset_test, batch_size, shuffle=False)

    return datalaoder_train, datalaoder_test

def calculate_correct_count(predicted, label):

    # Convert to boolean tensors for easy comparison
    predicted = predicted.to(torch.bool)
    label = label.to(torch.bool)

    type1_error = (~label & predicted).sum().item()             # False Positives (predicted=1, label=0)
    type1_error_count = predicted.sum().item()                  # Total predicted as Positive (predicted=1)

    return type1_error, type1_error_count

def predict_result(main_data, code, model, device, target, window_length, min_period=100):
    stock_data = main_data[main_data['code'] == code].tail(min_period).copy()
    data = process_stock_data(stock_data, target, window_length, predict=True)
    output = model(torch.tensor(data, dtype=torch.float32).unsqueeze(0).to(device))
    return round(output[0].cpu().detach().item(), 4)

def GetDatasetShape(main_data, window_length, code=1101, min_period=100):
    stock_data = main_data[main_data['code'] == code].tail(min_period).copy()
    stock_data = stock_data.drop(['code', 'date'], axis=1).reset_index(drop=True)
    stock_data = Set.Feature(stock_data)
    stock_data = stock_data.dropna()

    index_col_volume = stock_data.columns.get_loc("volume")

    stock_data = stock_data.to_numpy()

    return np.expand_dims(stock_data[-window_length:, index_col_volume + 1:], axis=0).shape

In [ ]:
# Select single Type to adjust the model and the optimizer
BATCH_SIZE = 256
N_EPOCHS = 20

# Type Id List [0, 7, 8, 13, 17, 18, 24, 25, 26, 27, 30, 31, 32, 35, 37]
type_id=27
target_pct = -0.05
threshold = 0.5

use_checkpoint = False
window_length = 20

list_unique_code_from_data = (SQLSentence.GetCodeByTypeId(type_id, connect))['code'].tolist()
datalaoder_train, datalaoder_test = build_dataloader(main_data, list_unique_code_from_data, target_pct, BATCH_SIZE, window_length)

model_name = 'CNN_LSTM'
ModelClass = getattr(Model, model_name)
model = ModelClass(datalaoder_test.dataset.shape).to(device)

lr = Set.GetInfo('learning_rate')
current_pos_weight  = math.log((Set.GetInfo('pos_weight'))[target_pct][dict_type_name[type_id]])+1
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([current_pos_weight]).to(device))
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

best_test_loss = Utility.load_best_model_record_txt(type_id, target_pct, model.__class__.__name__)
# best_test_loss = float('inf')

if use_checkpoint:
    try:
        postfix='save'
        model.load_state_dict(torch.load(f"{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_{postfix}", weights_only=True))
    except:
        print(f'No preserved model: {model.__class__.__name__}')

epoch = 0
while epoch < N_EPOCHS:
    epoch+=1
    print(f'Train {dict_type_name[type_id]} Epoch: {epoch}')

    # Train
    invoke_model('train', model, datalaoder_train, criterion, device, optimizer, threshold)

    # Test
    with torch.no_grad():
        test_avg_loss, test_error_ratio = invoke_model('test', model, datalaoder_test, criterion, device, threshold=threshold)

    # Save the model with a best loss (test) using postfix: best
    if test_avg_loss < best_test_loss:
        best_test_loss = test_avg_loss

        print(f'Model has saved in {PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_best\n')
        torch.save(model.state_dict(), f'{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_best')
        Utility.update_model_record_txt(type_id, target_pct, model.__class__.__name__, best_test_loss, test_error_ratio)

# Save the last epoch model using postfix: save
print(f'Model has saved in {PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_save\n')
torch.save(model.state_dict(), f'{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_save')

# del datalaoder_train, datalaoder_test

In [ ]:
# Predict
target_pct = 0.05
window_length = 20
threshold=0.5

# [0, 7, 8, 13, 17, 18, 24, 25, 26, 27, 30, 31, 32, 35, 37]
list_selected_type_id = Set.GetInfo('select_stock_list')
current_date = (SQLSentence.GetLatestDate(connect)).strftime("%Y-%m-%d")

model_name = 'CNN_LSTM'
ModelClass = getattr(Model, model_name)
postfix='best'

file_name = f'predict_tmp_{current_date}_{int(target_pct*100)}%_{model_name}_{postfix}.txt'

if not os.path.exists(file_name):
    with open(file_name, 'w') as f:
        pass

with open(file_name, 'a', encoding='utf-8') as f:
    f.write(f"----------------------------------Model: {model_name}_{postfix}----------------------------")

    for type_id in tqdm.tqdm(list_selected_type_id):

        list_unique_code_from_data = (SQLSentence.GetCodeByTypeId(type_id, connect))['code'].tolist()

        model = ModelClass(GetDatasetShape(main_data, window_length)).to(device)
        model.eval()
        list_code_investable = []

        try:
            model.load_state_dict(torch.load(f"{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_{postfix}", weights_only=True))
        except:
            print(f'{dict_type_name[type_id]} No preserved model: {model.__class__.__name__}')
            continue

        for code in list_unique_code_from_data:
            output = predict_result(main_data, code, model, device, target_pct, window_length)
            if output > threshold:
                list_code_investable.append((code, output))

        if len(list_code_investable) != 0:
            f.write("------------------------------------------------------------------------")
            f.write(f"\n產業 {type_id}: {dict_type_name[type_id]}\n")
            for code, probability in list_code_investable:
                f.write(f'{code} {dict_code_name[code]:<10} Prob: {probability}\n')
    f.write("\n\n")

In [ ]:
# Train all data
BATCH_SIZE = 256
N_EPOCHS = 20

target_pct = -0.03
window_length = 20
threshold=0.5
use_checkpoint = True

model_name = 'CNN_LSTM'
ModelClass = getattr(Model, model_name)

lr = Set.GetInfo('learning_rate')
list_selected_type_id = Set.GetInfo('select_stock_list')

for type_id in [0, 7, 8, 13, 17, 18]:
    list_unique_code_from_data = (SQLSentence.GetCodeByTypeId(type_id, connect))['code'].tolist()
    datalaoder_train, datalaoder_test = build_dataloader(main_data, list_unique_code_from_data, target_pct, BATCH_SIZE, window_length)

    model = ModelClass(datalaoder_train.dataset.shape).to(device)
    current_pos_weight  = math.log((Set.GetInfo('pos_weight'))[target_pct][dict_type_name[type_id]])+1
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([current_pos_weight]).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    
    test_avg_loss = 0
    best_test_loss = Utility.load_best_model_record_txt(type_id, target_pct, model.__class__.__name__)
    best_test_loss= float('inf')

    if use_checkpoint:
        try:
            postfix='best'
            model.load_state_dict(torch.load(f"{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_{postfix}", weights_only=True))
        except:
            print(f'{dict_type_name[type_id]} No preserved model: {model.__class__.__name__}')
            break

    epoch = 0
    while epoch < N_EPOCHS:
        epoch+=1
        print(f'Train {dict_type_name[type_id]} Epoch: {epoch}')

        # Train
        invoke_model('train', model, datalaoder_train, criterion, device, optimizer, threshold)

        # Test
        with torch.no_grad():
            test_avg_loss, test_error_ratio = invoke_model('test', model, datalaoder_test, criterion, device, threshold=threshold)

        # Save the model with a best loss (test) using postfix: best
        if test_avg_loss < best_test_loss:
            best_test_loss = test_avg_loss

            print(f'Model has saved in {PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_best')

            torch.save(model.state_dict(), f'{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_best')
            Utility.update_model_record_txt(type_id, target_pct, model.__class__.__name__, best_test_loss, test_error_ratio)

    # Save the last epoch model using postfix: save
    print(f'Model has saved in {PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_save\n')
    torch.save(model.state_dict(), f'{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_save')

    del datalaoder_train, datalaoder_test